In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import math
import io
from PIL import Image
from copy import deepcopy
from IPython.display import HTML
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split, Subset,ConcatDataset
import time
from matplotlib.colors import Normalize
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
import numpy as np

print(device)


In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
path='/home/scai/mtech/aib222683/scratch/Task2/data'
data_path_Train = os.path.join(path,'train')
img_path=[os.path.join(data_path_Train, filename) for filename in os.listdir(data_path_Train)]
original_dataset=[]
c=0
for i,data in enumerate(img_path):
    image = Image.open(data)
    to_tensor = transforms.ToTensor()
    image1=to_tensor(image)
    if i%3==0:
        original_dataset.append(image1)
    c+=1


In [ ]:
augmentation_transform = transforms.Compose([
    transforms.RandomRotation(25),
    # transforms.GaussianBlur(kernel_size=29, sigma=11),
    # transforms.RandomHorizontalFlip(),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    # transforms.RandomResizedCrop(224),
    # transforms.RandomVerticalFlip(),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

augmentation_transform_gaussian = transforms.Compose([
    # transforms.RandomRotation(45),
    transforms.GaussianBlur(kernel_size=29, sigma=9),
    # transforms.RandomHorizontalFlip(),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    # transforms.RandomResizedCrop(224),
    # transforms.RandomVerticalFlip(),
    # transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [ ]:
# log_path = os.path.join("/content/drive/MyDrive/Task1/logsaving_data") #Enter the log saving directory here
# /home/scai/mtech/aib222677/scratch/Task1/data

path1='/home/scai/mtech/aib222683/scratch/Task2/data'
# data_path_Train = os.path.join(path,'train') #Enter the train folder directory
# data_path_Test = os.path.join(path,'test') #Enter the test folder directory
data_path_Train = os.path.join(path1,'train')
data_path_Test = os.path.join(path1,'test')

# data_path_Train = os.path.dirname('/home/scai/mtech/aib222677/scratch/khushal/Task2/data/train')
# data_path_Test = os.path.dirname('/home/scai/mtech/aib222677/scratch/khushal/Task2/data/test')
batch_size = 16
num_workers = 2
transform_train = transforms.Compose([
                                # transforms.Resize((256,512)),
                                # transforms.RandomResizedCrop(size=(256,512)),
                                # transforms.RandomRotation(degrees=60),
                                # transforms.ElasticTransform(),
                                # transforms.ColorJitter(),
                                # transforms.GaussianBlur(kernel_size=5),
                                # transforms.Resize((256,512)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                                # transforms.functional.adjust_gamma(inpt, gamma[, gain]),
                                # transforms.GaussianBlur(kernel_size=5)
                                     ])
transform= transforms.Compose([
    # transforms.Resize((256,512)),
                                # transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])


In [ ]:
class BrainImage(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        self.image_paths = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, filename)) and not filename.startswith('.')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image


In [ ]:
class AugmentedImage(Dataset):
    def __init__(self, data,transform1=None,transform2=None,transform3=None):
        self.data = data
        self.transform1 =transform1
        self.transform2=transform2
        self.transform3=transform3

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image= self.data[index]


        random_number = random.randint(1, 3)
    
        if random_number == 1:
           augmented_image = self.transform1(image)
           # augmented_dataset.append(augmented_image)

            
        
        elif random_number == 2:
           img1=image[:,:,:256]
           img2=image[:,:,256:]
           img1=transforms.functional.adjust_gamma(img1,gamma=0.8,gain=1)
           normal_image=self.transform3(img1)
           normal_image2=self.transform3(img2)
           img=torch.cat((normal_image, normal_image2), dim=2)
           # augmented_dataset.append(img)
           image=img
            
        else:
            img1=image[:,:,:256]
            img2=image[:,:,256:]
            augmented_image = self.transform2(img2)
            img=torch.cat((augmented_image, img2), dim=2)
            image=img
            # augmented_dataset.append(img)


        return image

augmented_data=AugmentedImage(original_dataset,augmentation_transform,augmentation_transform_gaussian,transform)


In [ ]:
# Load the entire training dataset

full_dataset=BrainImage(data_dir=data_path_Train, transform=transform_train)
# c=0
# for data in full_dataset:
#     c+=1
#     print(type(data))
#     break
# print(len(full_dataset))
# Split the dataset into training and validation sets (use the first 1000 for training)
# train_dataset, _ = random_split(full_dataset, [1000, len(full_dataset) - 1000])
# train_dataset=Subset(full_dataset, range(5000))
# train_dataset=Subset(full_dataset, range(25000))   # 30 patients
# train_dataset=full_dataset

train_dataset=full_dataset+augmented_data
# Create a data loader for training
load_Train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [ ]:
# Load the entire test dataset
full_test_dataset = BrainImage(data_dir=data_path_Test, transform=transform_train)
test_dataset=full_test_dataset

In [ ]:
def split(img):
    return img[:,:,:,:256], img[:,:,:,256:]

In [ ]:
inst_norm = True if batch_size==1 else False  # instance normalization


def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    return nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride,
    padding=padding)


def conv_n(in_channels, out_channels, kernel_size, stride=1, padding=0, inst_norm=False):
    if inst_norm == True:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding), nn.InstanceNorm2d(out_channels,
        momentum=0.1, eps=1e-5),)
    else:
        return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding), nn.BatchNorm2d(out_channels,
        momentum=0.1, eps=1e-5),)

def tconv(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0,):
    return nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=stride,
    padding=padding, output_padding=output_padding)

def tconv_n(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, inst_norm=False):
    if inst_norm == True:
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding, output_padding=output_padding),
        nn.InstanceNorm2d(out_channels, momentum=0.1, eps=1e-5),)
    else:
        return nn.Sequential(nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
        stride=stride, padding=padding, output_padding=output_padding),
        nn.BatchNorm2d(out_channels, momentum=0.1, eps=1e-5),)

In [ ]:
dim_c = 3
dim_g = 64

# Generator
class Gen(nn.Module):
    def __init__(self, inst_norm=False):
        super(Gen,self).__init__()
        self.n1 = conv(dim_c, dim_g, 4, 2, 1)
        self.n2 = conv_n(dim_g, dim_g*2, 4, 2, 1, inst_norm=inst_norm)
        self.n3 = conv_n(dim_g*2, dim_g*4, 4, 2, 1, inst_norm=inst_norm)
        self.n4 = conv_n(dim_g*4, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n5 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n6 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n7 = conv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.n8 = conv(dim_g*8, dim_g*8, 4, 2, 1)

        self.m1 = tconv_n(dim_g*8, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m2 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m3 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m4 = tconv_n(dim_g*8*2, dim_g*8, 4, 2, 1, inst_norm=inst_norm)
        self.m5 = tconv_n(dim_g*8*2, dim_g*4, 4, 2, 1, inst_norm=inst_norm)
        self.m6 = tconv_n(dim_g*4*2, dim_g*2, 4, 2, 1, inst_norm=inst_norm)
        self.m7 = tconv_n(dim_g*2*2, dim_g*1, 4, 2, 1, inst_norm=inst_norm)
        self.m8 = tconv(dim_g*1*2, dim_c, 4, 2, 1)
        self.tanh = nn.Tanh()

    def forward(self,x):
        n1 = self.n1(x)
        n2 = self.n2(F.leaky_relu(n1, 0.2))
        n3 = self.n3(F.leaky_relu(n2, 0.2))
        n4 = self.n4(F.leaky_relu(n3, 0.2))
        n5 = self.n5(F.leaky_relu(n4, 0.2))
        n6 = self.n6(F.leaky_relu(n5, 0.2))
        n7 = self.n7(F.leaky_relu(n6, 0.2))
        n8 = self.n8(F.leaky_relu(n7, 0.2))
        m1 = torch.cat([F.dropout(self.m1(F.relu(n8)), 0.5, training=True), n7], 1)
        m2 = torch.cat([F.dropout(self.m2(F.relu(m1)), 0.5, training=True), n6], 1)
        m3 = torch.cat([F.dropout(self.m3(F.relu(m2)), 0.5, training=True), n5], 1)
        m4 = torch.cat([self.m4(F.relu(m3)), n4], 1)
        m5 = torch.cat([self.m5(F.relu(m4)), n3], 1)
        m6 = torch.cat([self.m6(F.relu(m5)), n2], 1)
        m7 = torch.cat([self.m7(F.relu(m6)), n1], 1)
        m8 = self.m8(F.relu(m7))

        return self.tanh(m8)

In [ ]:
dim_d = 64

# Discriminator
class Disc(nn.Module):
    def __init__(self, inst_norm=False):
        super(Disc,self).__init__()
        self.c1 = conv(dim_c*2, dim_d, 4, 2, 1)
        self.c2 = conv_n(dim_d, dim_d*2, 4, 2, 1, inst_norm=inst_norm)
        self.c3 = conv_n(dim_d*2, dim_d*4, 4, 2, 1, inst_norm=inst_norm)
        self.c4 = conv_n(dim_d*4, dim_d*8, 4, 1, 1, inst_norm=inst_norm)
        self.c5 = conv(dim_d*8, 1, 4, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, y):
        xy=torch.cat([x,y],dim=1)
        xy=F.leaky_relu(self.c1(xy), 0.2)
        xy=F.leaky_relu(self.c2(xy), 0.2)
        xy=F.leaky_relu(self.c3(xy), 0.2)
        xy=F.leaky_relu(self.c4(xy), 0.2)
        xy=self.c5(xy)

        return self.sigmoid(xy)

def weights_init(z):
    cls_name =z.__class__.__name__
    if cls_name.find('Conv')!=-1 or cls_name.find('Linear')!=-1:
        nn.init.normal_(z.weight.data, 0.0, 0.02)
        nn.init.constant_(z.bias.data, 0)
    elif cls_name.find('BatchNorm')!=-1:
        nn.init.normal_(z.weight.data, 1.0, 0.02)
        nn.init.constant_(z.bias.data, 0)

In [ ]:
BCE = nn.BCELoss() #binary cross-entropy
L1 = nn.L1Loss()
L2=nn.MSELoss()
#instance normalization
Gen_model = Gen(inst_norm).to(device)
Disc = Disc(inst_norm).to(device)
generator = Gen(inst_norm).to(device)
# #optimizers
# Gen_optim = optim.Adam(Gen.parameters(), lr=2e-4, betas=(0.5, 0.999), weight_decay=0.35)
Disc_optim = optim.Adam(Disc.parameters(), lr=5e-5, betas=(0.5, 0.999))
Gen_optim = optim.Adam(Gen_model.parameters(), lr=1e-4, betas=(0.5, 0.999))

In [ ]:
# fix_con, _ = next(iter(load_Test))|
fix_con= next(iter(load_Train))
fix_con = fix_con.to(device)
fix_X, fix_y = split(fix_con)

def compare_batches(batch1, batch2, batch3, title1, title2,title3):
    # batch1
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.axis("off")
    plt.title(title1)
    plt.imshow(np.transpose(vutils.make_grid(batch1, nrow=1, padding=5,
    normalize=True).cpu(), (1,2,0)))
    # batch2
    plt.subplot(1,3,2)
    plt.axis("off")
    plt.title(title2)
    plt.imshow(np.transpose(vutils.make_grid(batch2, nrow=1, padding=5,
    normalize=True).cpu(), (1,2,0)))
    # third batch
    if batch3 is not None:
        plt.subplot(1,3,3)
        plt.axis("off")
        plt.title(title3)
        plt.imshow(np.transpose(vutils.make_grid(batch3, nrow=1, padding=5,
        normalize=True).cpu(), (1,2,0)))

with torch.no_grad():
    fk = Gen_model(fix_X)
# compare_batches(fix_X, fk,fix_y, "input image", "prediction",  "ground truth")
for j in range(fk.shape[0]):
    if j%3==0:
        figs=plt.figure(figsize=(10,10))

        plt.subplot(1,3,1)
        plt.axis("off")
        plt.title("input image")
        plt.imshow(np.transpose(vutils.make_grid(fix_X[j], nrow=1, padding=5,
        normalize=True).cpu(), (1,2,0)))
    
        plt.subplot(1,3,2)
        plt.axis("off")
        plt.title("generated image")
        plt.imshow(np.transpose(vutils.make_grid(fk[j], nrow=1, padding=5,
        normalize=True).cpu(), (1,2,0)))
    
        plt.subplot(1,3,3)
        plt.axis("off")
        plt.title("ground truth")
        plt.imshow(np.transpose(vutils.make_grid(fix_y[j], nrow=1, padding=5,
        normalize=True).cpu(), (1,2,0)))


In [ ]:
# # img_list = []
# # Disc_losses = Gen_losses = Gen_GAN_losses = Gen_L1_losses = []


iter_per_plot = 20
epochs = 100
L1_lambda = 50.0
# ssim_scores = []
# mse_scores = []

for ep in range(epochs):
    for i, data in enumerate(load_Train):
        start_time = time.time()
        size = data.shape[0]

        x, y = split(data.to(device))

        r_masks = torch.ones(size,1,30,30).to(device)
        f_masks = torch.zeros(size,1,30,30).to(device)

        # disc
        Disc.zero_grad()
        #real_patch
        r_patch=Disc(y,x)
        r_gan_loss=L2(r_patch,r_masks)

        fake=Gen_model(x)
        #fake_patch
        f_patch = Disc(fake.detach(),x)
        f_gan_loss=L2(f_patch,f_masks)

        Disc_loss = r_gan_loss + f_gan_loss
        Disc_loss.backward()
        Disc_optim.step()

        # gen
        Gen_model.zero_grad()
        f_patch = Disc(fake,x)
        f_gan_loss=L2(f_patch,r_masks)

        L1_loss = L1(fake,y)
        Gen_loss = f_gan_loss + L1_lambda*L1_loss
        Gen_loss.backward()

        Gen_optim.step()
        end_time = time.time()  # End measuring time for each iteration
        elapsed_time = end_time - start_time  # Calculate elapsed time

        # Print custom training information
        # print('(epoch: {:d}, iters: {:d}, time: {:.3f}, data: 0.003) G_GAN: {:.3f} G_L1: {:.3f} D_real: {:.3f} D_fake: {:.3f}'.format(ep, i, elapsed_time, Gen_GAN_losses[-1], Gen_L1_losses[-1], r_patch.mean(), f_patch.mean()))

        if (i+1)%iter_per_plot == 0 :

            print('Epoch [{}/{}], Step [{}/{}], disc_loss: {:.4f}, gen_loss: {:.4f},Disc(real): {:.2f}, Disc(fake):{:.2f}, gen_loss_gan:{:.4f}, gen_loss_L1:{:.4f}'.format(ep+1, epochs, i+1, len(load_Train), Disc_loss.item(), Gen_loss.item(), r_patch.mean(), f_patch.mean(), f_gan_loss.item(), L1_loss.item()))

            # Gen_losses.append(Gen_loss.item())
            # Disc_losses.append(Disc_loss.item())
            # Gen_GAN_losses.append(f_gan_loss.item())
            # Gen_L1_losses.append(L1_loss.item())
    if (ep+1)%5==0:
        with torch.no_grad():
            Gen_model.eval()
            for data in load_Train:
                # print(data.shape)
                x, y = split(data.to(device))
                fake = Gen_model(x)
                for j in range(fake.shape[0]):
                    if j%10==0:
                        figs=plt.figure(figsize=(10,10))
            
                        plt.subplot(1,3,1)
                        plt.axis("off")
                        plt.title("input image")
                        plt.imshow(np.transpose(vutils.make_grid(x[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
                    
                        plt.subplot(1,3,2)
                        plt.axis("off")
                        plt.title("generated image")
                        plt.imshow(np.transpose(vutils.make_grid(fake[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
                    
                        plt.subplot(1,3,3)
                        plt.axis("off")
                        plt.title("ground truth")
                        plt.imshow(np.transpose(vutils.make_grid(y[j], nrow=1, padding=5,
                        normalize=True).cpu(), (1,2,0)))
        
                    
                # compare_batches(x, fake, y,"input images", "predicted images",  "ground truth")
                break
            Gen_model.train()
        
        if ep+1==epochs:
            torch.save(Gen_model.state_dict(), 'model18.pth') 

In [ ]:
batch_size = 1  # Set the batch size to 1 to get a single image in each iteration
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
c=0
generator.load_state_dict(torch.load('model18.pth', map_location=device))

for i,data in enumerate (test_loader):
    if i%155==0:
        c+=1
        t_batch=data
        t_x, t_y = split(t_batch)
        
        with torch.no_grad():
            generator.eval()
            fk= generator(t_x.to(device))
            t_y=t_y.to(device)
            figs=plt.figure(figsize=(10,10))

            plt.subplot(1,4,1)
            plt.axis("off")
            plt.title("input image")
            
            plt.imshow(np.rot90(np.transpose(vutils.make_grid(t_x[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0)),k=-1))
            
            
            plt.subplot(1,4,2)
            plt.axis("off")
            plt.title("generated image")
            plt.imshow(np.rot90(np.transpose(vutils.make_grid(fk[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0)),k=-1))
            # plt.colorbar()
            
            plt.subplot(1,4,3)
            plt.axis("off")
            plt.title("ground truth")
            plt.imshow(np.rot90(np.transpose(vutils.make_grid(t_y[0], nrow=1, padding=5,
            normalize=True).cpu(), (1,2,0)),k=-1))
            # plt.colorbar()
            # print(t_y[j].is_cuda)
            # print(fk[j].is_cuda)
            plt.subplot(1,4,4)
            plt.axis("off")
            plt.title("error map")
    
        
            data_eg=np.transpose(np.abs((t_y[0]-fk[0]).cpu().numpy()),(1,2,0))
            normalized_data=Normalize()(data_eg)
            normalized_data=np.mean(normalized_data, axis=2, keepdims=True)
            # print(normalized_data.shape)
            plt.imshow(np.rot90((normalized_data),k=-1),cmap='jet')

            if c==10:
                break
                

In [ ]:
def calculate_psnr(true_image, fake_image, data_range):
    mse = np.mean((true_image - fake_image) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * np.log10(data_range / np.sqrt(mse))
    return psnr

batch_size = 1  # Set the batch size to 1 to get a single image in each iteration
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

error_list = []
structural_similarity = []
psnr_list = []

# Now you can iterate over it with random samples
for i, data in enumerate(test_loader):
    t_batch = data
    t_x, t_y = split(t_batch)
    with torch.no_grad():
        generator.eval()
        fk_batch = generator(t_x.to(device))
    # if i%10==0:
    #     compare_batches(t_x, fk_batch, t_y, "input images", "predicted images","ground truth")
    t_y = t_y.numpy()
    fk_batch = fk_batch.cpu().detach().numpy()

    # Calculate RMSE
    rmse = np.sqrt(np.mean((t_y - fk_batch) ** 2))
    error_list.append(rmse)

    # Calculate SSIM
    t_y = np.transpose(t_y.squeeze(), (1, 2, 0))
    fk_batch = np.transpose(fk_batch.squeeze(), (1, 2, 0))
    ssi = ssim(t_y, fk_batch, multichannel=True, win_size=3, data_range=2.0)
    structural_similarity.append(ssi)

    # Calculate PSNR
    psnr_value = calculate_psnr(t_y, fk_batch, data_range=2.0)
    psnr_list.append(psnr_value)

# Convert lists to numpy arrays for statistics calculation
error_list = np.array(error_list)
structural_similarity_np = np.array(structural_similarity)
psnr_np = np.array(psnr_list)

# Print statistics
print('Mean RMSE error:', np.mean(error_list))
print('Max RMSE error:', np.max(error_list))
print('Min RMSE error:', np.min(error_list))
print('Mean SSIM:', np.mean(structural_similarity_np))
print('Min SSIM:', np.min(structural_similarity_np))
print('Max SSIM:', np.max(structural_similarity_np))
print('Mean PSNR:', np.mean(psnr_np))
print('Min PSNR:', np.min(psnr_np))
print('Max PSNR:', np.max(psnr_np))